In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation
from tensorflow.keras.layers import Flatten,Conv2D,MaxPooling2D
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard
import time
import pickle
import numpy as np  #to run array operations
import matplotlib.pyplot as plt   # to plot the or to show the images
import os    #to iterate throgh directories
import cv2 
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K
K.image_data_format()

'channels_last'

In [31]:
path_test = 'E:/oct images/OCT2017_/train'
CATEGORIES = ['CNV','DME','DRUSEN','NORMAL']
IMG_SIZE = 224

In [32]:
training = []
def createTrainingData():
    for category in CATEGORIES:
        path = os.path.join(path_test, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            img_array = cv2.imread(os.path.join(path,img))
            next_arr=cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
            new_array = cv2.resize(next_arr, (IMG_SIZE, IMG_SIZE))
            training.append([new_array, class_num])
createTrainingData()

In [9]:
import random
random.shuffle(training)

In [33]:
X =[]
y =[]
for features, label in training:
    X.append(features)
    y.append(label)
#X = np.array(X).reshape(-1,IMG_SIZE,IMG_SIZE,1)

In [39]:
X.shape

(224, 224)

In [2]:
X = np.array(X)
X = X.astype('float32')
X/=255

In [12]:
num_channel=1
if num_channel==1:
    if K.image_data_format()=='th':
        X= np.expand_dims(X, axis=1)
        print (X.shape)
    else:
        X= np.expand_dims(X, axis=3)
        print (X.shape)

else:
    if K.image_data_format()=='th':
        X = np.rollaxis(X,3,1)
        print (X.shape)

(83484, 128, 128, 1)


In [13]:
from tensorflow.keras.utils import to_categorical
Y = to_categorical(y, num_classes=4)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 4)

In [ ]:
# batch_size = 16
# nb_classes =4
# nb_epochs = 5
# img_rows, img_columns = 200, 200
# img_channel = 3
# nb_filters = 32
# nb_pool = 2
# nb_conv = 3

In [10]:
model = Sequential()

model.add(Conv2D(32, (3,3), activation = 'relu',   padding = 'same',input_shape = (128,128,1)))
model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))

model.add(Conv2D(64, (3,3), activation = 'relu', padding = 'same'))
model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))

model.add(Conv2D(128, (3,3), activation='relu', padding = 'same'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(256, (3,3), activation='relu', padding = 'same'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(512, (3,2), activation='relu', padding = 'same'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Flatten())

# model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))

model.add(Dense(4, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 32)      320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 256)       2

In [26]:
from tensorflow.keras.optimizers import SGD,RMSprop,Adam

In [12]:
learning_rate = 0.0001
opt = Adam(lr = learning_rate) 
model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'] )

In [ ]:
hist = model.fit(X_train, y_train, batch_size = 200, epochs = 20, verbose = 1, validation_split = 0.2)

In [15]:
acc_score = model.evaluate(X_test,y_test)
print(acc_score[0])
print(acc_score[1]*100)

522/522 [==============================] - 71s 133ms/step - loss: 0.2534 - accuracy: 0.9122
0.2534400224685669
0.9121997952461243


In [16]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [3]:
tf.keras.models.model_from_json

<function tensorflow.python.keras.saving.model_config.model_from_json(json_string, custom_objects=None)>

In [4]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = tf.keras.models.model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [23]:
model1 = tf.keras.applications.densenet.DenseNet121(
    include_top=True, weights='imagenet', input_tensor=None,
    input_shape=None
)

In [21]:
json_file = open('model.json', 'r')
loaded_model_json1 = json_file.read()
json_file.close()
loaded_model1 = tf.keras.models.model_from_json(loaded_model_json1)

In [24]:
# loaded_model1.load_weights('densenet121_weights_tf_dim_ordering_tf_kernels.h5')
model1.summary()

Model: "densenet121"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_2 (ZeroPadding2D (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_2[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
________________________________________________________________________________________

In [27]:
learning_rate = 0.0001
opt = Adam(lr = learning_rate) 
model1.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'] )

In [1]:
hist = model1.fit(X_train, y_train, batch_size = 200, epochs = 10, verbose = 1, validation_split = 0.2)

In [29]:
# model1.evaluate(X_test,y_test)